In [1]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# CSV öffnen: Grundstück-Daten GIS

gis = pd.read_csv('Grundstuecke_Chur.csv', sep=';')


In [3]:
# GIS bereinigen

# überflüssige Spalten löschen
gis.pop('3072')
gis.pop('Grundstueck-Koordinate X')
gis.pop('Grundstueck-Koordinate Y')
gis.pop('Grundstueck-Ort')
gis.pop('Grundstueck-Gemeinde')

# Zeilen mit NaN in Grundstueck-Nummer löschen
gis = gis.dropna(axis=0, subset=['Grundstueck-Nummer'])



# Grundstueck-Nummer = in int ändern

gis['Grundstueck-Nummer'] = gis['Grundstueck-Nummer'].astype(int)

# zusätzliche Spalte Vor- und Nachname
gis = gis.fillna('')
gis['NameVorname'] = gis['Name'] + " " + gis['Vorname']


# Reihenfolge der Spalten ändern
gis = gis[['Grundstueck-Nummer', 'Grundstueck-Typ', 'Grundstueck-Flaeche', 'Eigentumsart', 'Name', 'Vorname', 'NameVorname', 'Adresse', 'PLZ', 'Ort', 'Datum der Abfrage']]



In [4]:
# Sortieren nach Grundstück-Nummer
gis.sort_values("Grundstueck-Nummer").head(10)



,Grundstueck-Nummer,Grundstueck-Typ,Grundstueck-Flaeche,Eigentumsart,Name,Vorname,NameVorname,Adresse,PLZ,Ort,Datum der Abfrage
7621,1,Liegenschaft,4309.0,Alleineigentum,Rhätische Bahn AG (RhB),,Rhätische Bahn AG (RhB),Bahnhofstrasse 25,7001,Chur,2019-12-28 12:55:50.879251
8781,2,Liegenschaft,669.0,Alleineigentum,Anlagestiftung Swiss Life,,Anlagestiftung Swiss Life,General-Guisan-Quai 40,8002,Zürich Zustellkreis,2019-12-28 12:56:28.368409
9074,3,Liegenschaft,1397.0,Alleineigentum,Rhätische Bahn AG (RhB),,Rhätische Bahn AG (RhB),Bahnhofstrasse 25,7001,Chur,2019-12-28 12:57:18.084170
5225,7,Liegenschaft,1065.0,Alleineigentum,Stadt Chur,,Stadt Chur,,,,2019-12-28 12:58:05.130008
6547,11,Liegenschaft,3990.0,Alleineigentum,Stadt Chur,,Stadt Chur,,,,2019-12-28 12:58:59.382967
5474,12,Liegenschaft,3505.0,Alleineigentum,Stadt Chur,,Stadt Chur,,,,2019-12-28 12:59:44.221627
5207,13,Liegenschaft,237.0,Miteigentum,Arpagaus,Rita,Arpagaus Rita,Vogelsangweg 21,7000,Chur,2019-12-28 13:00:34.689663
5206,13,Liegenschaft,237.0,Miteigentum,Arpagaus,Renato,Arpagaus Renato,Vogelsangweg 22,7000,Chur,2019-12-28 13:00:34.564304
5205,13,Liegenschaft,237.0,Miteigentum,Arpagaus,Guido,Arpagaus Guido,Vogelsangweg 23,7000,Chur,2019-12-28 13:00:34.445602
5204,13,Liegenschaft,237.0,Miteigentum,Arpagaus,Anita Martina Maria,Arpagaus Anita Martina Maria,Quaderstrasse 36,7000,Chur,2019-12-28 13:00:34.329489


In [5]:
# CSV öffnen: EGID-Daten

egid = pd.read_csv('Gebäude_Chur.csv', sep=';')
egid.head()

,Grundstueck-Nummer,EGID,Gebaeude-Koordinate X,Gebaeude-Koordinate Y,Gebaeude-Baujahr,Gebaeude-Kategorie,Gebaeude-Klasse-Nummer,Gebaeude-Klasse,Gebaeude-Flaeche,Gebaeude-Geschosse,Gebaeude-Strasse,Gebaeude-Hausnummer,Gebaeude-PLZ,Gebaeude-Ort
0,1764.0,001 739 400,2758845,1190923,1901,"Mehrfamilienhaus, ohne Nebennutzung",1122,Gebäude mit drei oder mehr Wohnungen,92,3,Kettweg,8,7000,Chur
1,1869.0,191 204 837,2758393,1191136,2016,"Mehrfamilienhaus, ohne Nebennutzung",1122,Gebäude mit drei oder mehr Wohnungen,,5,Pulvermühlestrasse,34,7000,Chur
2,20687.0,001 210 517,2761752,1189391,,,Anz. separate Wohnräume,,,2,Dorfstrasse,30,7026,Maladers
3,4899.0,001 739 372,2758712,1190824,1959,"Mehrfamilienhaus, ohne Nebennutzung",1122,Gebäude mit drei oder mehr Wohnungen,110,3,Salvatorenstrasse,52,7000,Chur
4,7261.0,009 021 636,2760625,1188709,1995,Nebennutzung Gebäude mit zwei Wohnungen,Anz. separate Wohnräume,88 1,Gebäude mit zwei Wohnungen,2,Kronenhofweg,18,7000,Chur


In [6]:
# egid bereinigen

# überflüssige Spalten löschen
egid.pop('Gebaeude-Baujahr')
egid.pop('Gebaeude-Kategorie')
egid.pop('Gebaeude-Klasse-Nummer')
egid.pop('Gebaeude-Klasse')
egid.pop('Gebaeude-Flaeche')
egid.pop('Gebaeude-Geschosse')

# Zeilen mit NaN in Grundstueck-Nummer löschen
egid = egid.dropna(axis=0, subset=['Grundstueck-Nummer'])

egid = egid.fillna('')


# Grundstueck-Nummer = in int ändern

egid['Grundstueck-Nummer'] = egid['Grundstueck-Nummer'].astype(int)
egid['Gebaeude-PLZ'] = egid['Gebaeude-PLZ'].astype(int)
egid['Gebaeude-PLZ'] = egid['Gebaeude-PLZ'].astype(str)






In [7]:
# Sortieren nach Grundstück-Nummer
egid.sort_values("Grundstueck-Nummer").tail(5)

,Grundstueck-Nummer,EGID,Gebaeude-Koordinate X,Gebaeude-Koordinate Y,Gebaeude-Strasse,Gebaeude-Hausnummer,Gebaeude-PLZ,Gebaeude-Ort
2650,20816,003 035 276,2761907,1189486,Hinder der Chilcha,15,7026,Maladers
113,20817,190 165 553,2761722,1189495,Under Quadera,4,7026,Maladers
2024,20818,190 010 624,2761890,1189507,Oberdorfstrasse,22,7026,Maladers
4793,20821,190 155 477,2761667,1189266,Wingertweg,31,7026,Maladers
1828,20838,190 639 770,2761737,1189233,Under Matätsch,7,7026,Maladers


In [8]:
egid.sort_values("Gebaeude-Koordinate X").tail(5)

,Grundstueck-Nummer,EGID,Gebaeude-Koordinate X,Gebaeude-Koordinate Y,Gebaeude-Strasse,Gebaeude-Hausnummer,Gebaeude-PLZ,Gebaeude-Ort
2150,20284,001 210 605,2763002,1189758,Sa,12,7026,Maladers
2492,3587,001 741 053,x2758893,1191692,Lerchenweg,7,7000,Chur
2296,4620,001 741 091,x2758904,1191545,Sennensteinstrasse,19,7000,Chur
2513,4620,001 741 090,x2758905,1191545,Sennensteinstrasse,17,7000,Chur
2475,10108,001 741 696,x2758910,1191807,Birkenweg,2,7000,Chur


In [9]:
# fehlerhafte X-Koordinaten bereinigen: X-Werte, beginnend mit "x" (warum auch immer...)

#egid.loc[XXXX, 'Gebaeude-Koordinate X'] = 2758893  (XXXX ist der Indexwert dieser Zeile)


egid.loc[2492, 'Gebaeude-Koordinate X'] = '2758893'
egid.loc[2296, 'Gebaeude-Koordinate X'] = '2758904'
egid.loc[2513, 'Gebaeude-Koordinate X'] = '2758905'
egid.loc[2475, 'Gebaeude-Koordinate X'] = '2758910'

egid['Gebaeude-Koordinate X'] = egid['Gebaeude-Koordinate X'].astype(int)

egid.dtypes

Grundstueck-Nummer        int64
EGID                     object
Gebaeude-Koordinate X     int64
Gebaeude-Koordinate Y     int64
Gebaeude-Strasse         object
Gebaeude-Hausnummer      object
Gebaeude-PLZ             object
Gebaeude-Ort             object
dtype: object

In [11]:
# Geodataframe erstellen

# Koordinaten respektive Punkte in neue Spalte
egid['Koordinate'] = egid.apply(lambda row: Point(row['Gebaeude-Koordinate X'], row['Gebaeude-Koordinate Y']), axis=1)


# eigentliches GeoDataFrame
egid = gpd.GeoDataFrame(egid, geometry='Koordinate')


In [12]:
# egid und gis mergen


houses = egid.merge(gis, how='left', on='Grundstueck-Nummer')

# nur Besitzer des Hauses
haus = houses[houses['Grundstueck-Typ'] == 'Liegenschaft']

# nur Baurechtsnehmer des Hauses
baurecht = houses[houses['Grundstueck-Typ'] == 'SelbstRecht, Baurecht']

# nur Grundstück-Besitzer (ob mit oder ohne Gebäude drauf), aber OHNE Koordinaten
grundstueck = gis[gis['Grundstueck-Typ'] == "Liegenschaft"]


In [13]:
grundstueck

,Grundstueck-Nummer,Grundstueck-Typ,Grundstueck-Flaeche,Eigentumsart,Name,Vorname,NameVorname,Adresse,PLZ,Ort,Datum der Abfrage
0,20015,Liegenschaft,87.0,Alleineigentum,Hug,Claudio,Hug Claudio,Arosastrasse 15,7026,Maladers,2020-01-14 22:27:24.452206
2,3814,Liegenschaft,708.0,Alleineigentum,Stiftung Kantonsspital Graubünden,,Stiftung Kantonsspital Graubünden,Loëstrasse 170,7000,Chur,2020-01-10 06:15:36.154510
3,20773,Liegenschaft,108.0,Alleineigentum,Müller,Beda (verbeiratet),Müller Beda (verbeiratet),Churerweg 39,7026,Maladers,2020-01-15 01:43:03.125015
4,20767,Liegenschaft,858.0,Alleineigentum,Hassler,Anton,Hassler Anton,Schloss 48,7026,Maladers,2020-01-15 01:41:30.871202
5,4809,Liegenschaft,1058.0,Miteigentum,Quinter,Diego,Quinter Diego,Prasserieweg 12,7000,Chur,2020-01-10 21:10:05.676884
...,...,...,...,...,...,...,...,...,...,...,...
18707,20018,Liegenschaft,549.0,Alleineigentum,Berger,Peter,Berger Peter,Tumma 126A,7026,Maladers,2020-01-14 22:28:10.378124
18708,4810,Liegenschaft,1443.0,Alleineigentum,Stadt Chur,,Stadt Chur,,,,2020-01-10 21:10:23.581426
18709,4804,Liegenschaft,1237.0,Alleineigentum,Hartmann,Jürg,Hartmann Jürg,Prasserieweg 10,7000,Chur,2020-01-10 21:09:01.287635
18711,6975,Liegenschaft,379.0,Miteigentum,Lang,Eduard,Lang Eduard,Brändligasse 18,7000,Chur,2019-12-30 04:39:55.996836


In [14]:
# DataFrames in csv abspeichern


# Hausbesitzer mit Koordinaten
haus.to_csv('Hausbesitzer_mit_Koordinaten.csv', sep=";", index=False)

# nur Baurechtsnehmer mit Koordinaten
baurecht.to_csv('Baurechtsnehmer_mit_Koordinaten.csv', sep=";", index=False)

# nur Grundstück-Besitzer (ob mit oder ohne Gebäude drauf), aber OHNE Koordinaten
grundstueck.to_csv('Liegenschaftsbesitzer_ohne_Koordinaten.csv', sep=";", index=False)

